<a href="https://colab.research.google.com/github/sid0312/10DaysMLChallenge/blob/master/conusmer_complaints_using_tfdif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('/content/drive/My Drive/consumer_complaints.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [5]:
col = ['product', 'consumer_complaint_narrative']
df = df[col]
df = df[pd.notnull(df['consumer_complaint_narrative'])]
df.columns = ['product', 'consumer_complaint_narrative']
df['category_id'] = df['product'].factorize()[0]
category_id_df = df[['product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'product']].values)
df.head()

,product,consumer_complaint_narrative,category_id
190126,Debt collection,XXXX has claimed I owe them {$27.00} for XXXX ...,0
190135,Consumer Loan,Due to inconsistencies in the amount owed that...,1
190155,Mortgage,In XX/XX/XXXX my wages that I earned at my job...,2
190207,Mortgage,I have an open and current mortgage with Chase...,2
190208,Mortgage,XXXX was submitted XX/XX/XXXX. At the time I s...,2


In [6]:
df.shape

(66806, 3)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
dfx=df[:10000]

In [8]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(dfx.consumer_complaint_narrative).toarray()
labels = dfx.category_id
features.shape

(10000, 26806)

In [9]:
features[0]

array([0.10025273, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

In [10]:
dfx.shape

(10000, 3)

In [11]:
dfx.head()

,product,consumer_complaint_narrative,category_id
190126,Debt collection,XXXX has claimed I owe them {$27.00} for XXXX ...,0
190135,Consumer Loan,Due to inconsistencies in the amount owed that...,1
190155,Mortgage,In XX/XX/XXXX my wages that I earned at my job...,2
190207,Mortgage,I have an open and current mortgage with Chase...,2
190208,Mortgage,XXXX was submitted XX/XX/XXXX. At the time I s...,2


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [0]:
X_train,X_test,y_train,y_test = train_test_split(dfx['consumer_complaint_narrative'],dfx['product'],random_state=42,shuffle=True,test_size=0.3)

In [14]:
X_train.shape

(7000,)

In [0]:
cv  = CountVectorizer()

In [0]:
X_count = cv.fit_transform(X_train)
transformer = TfidfTransformer()
X_trans = transformer.fit_transform(X_count)

model =  MultinomialNB().fit(X_trans,y_train)

In [0]:
dfy = df[10000:]

In [21]:
dfy.shape

(56806, 3)

In [22]:
dfy.head()

,product,consumer_complaint_narrative,category_id
232613,Mortgage,I am unable to pay more mortgage for financial...,2
232615,Debt collection,They will not tell me who the original credito...,0
232616,Mortgage,Mortgage Escrow account we asked Chase to rem...,2
232619,Mortgage,My loan was a XXXX XXXX loan and was a 10 year...,2
232621,Debt collection,Toyota financial repeatedly calling all hours ...,0


In [0]:
cs_narrative_test = list(dfy['consumer_complaint_narrative'])

In [28]:
cs_narrative_test[0]

'I am unable to pay more mortgage for financial hardship. I have been in close contact with my lender since XX/XX/XXXX and filled out paperwork for a deed in Lieu of Foreclosure. A month after submitting this paperwork, we received notification from our lender that our XXXX loan on the property was transferred to another lender ( prior to this Ocwen, our lender, held XXXX loans ). The Deed in Lieu was rejected on account of this XXXX lien on the property, Ocwen had created. The lender told me if the lien was cleared I could avoid foreclosure with a deed in lieu. Friends helped me pay off the XXXX lien in XX/XX/XXXX. I then resubmitted the Deed in XXXX paperwork ( in XX/XX/XXXX ). I also sent a letter from the XXXX lender which stated that my loan was paid in full, and in XX/XX/XXXX I sent a copy of the " Certificate of Discharge \'\' from my county courthouse. I have keep my relationship manager at Ocwen abreast of all of this and she acknowledge receiving all these documents. Then the

In [29]:
cs_label = list(dfy['product'])
cs_label[0]

'Mortgage'

In [31]:
print(model.predict(cv.transform([cs_narrative_test[0]])))

['Mortgage']


In [32]:
print(model.predict(cv.transform([cs_narrative_test[1]])))

['Debt collection']


In [36]:
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [38]:
cv_df

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.4405
1,RandomForestClassifier,1,0.4435
2,RandomForestClassifier,2,0.4350
3,RandomForestClassifier,3,0.4260
4,RandomForestClassifier,4,0.4205
5,MultinomialNB,0,0.6820
6,MultinomialNB,1,0.6740
7,MultinomialNB,2,0.6755
8,MultinomialNB,3,0.6730
9,MultinomialNB,4,0.6790


In [39]:
print(cv_df.groupby('model_name').accuracy.mean())

model_name
LogisticRegression        0.8137
MultinomialNB             0.6767
RandomForestClassifier    0.4331
Name: accuracy, dtype: float64
